In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the survey lung cancer.csv.
import pandas as pd
lung_cancer_df = pd.read_csv("survey lung cancer.csv")
lung_cancer_df.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2,YES
1,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2,YES
2,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2,NO
3,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2,NO
4,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1,NO


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
lung_cancer_df.drop(columns=['GENDER', 'AGE'], inplace=True)

In [3]:
# Determine the number of unique values in each column.
print(lung_cancer_df.nunique())

SMOKING                  2
YELLOW_FINGERS           2
ANXIETY                  2
PEER_PRESSURE            2
CHRONIC DISEASE          2
FATIGUE                  2
ALLERGY                  2
WHEEZING                 2
ALCOHOL CONSUMING        2
COUGHING                 2
SHORTNESS OF BREATH      2
SWALLOWING DIFFICULTY    2
CHEST PAIN               2
LUNG_CANCER              2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
print(lung_cancer_df['CHEST PAIN'].value_counts())

2    172
1    137
Name: CHEST PAIN, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 500
application_counts = lung_cancer_df['CHEST PAIN'].value_counts()
application_types_to_replace = list(application_counts[application_counts < cutoff].index)

# Replace in dataframe
for app in application_types_to_replace:
    lung_cancer_df['CHEST PAIN'] = lung_cancer_df['CHEST PAIN'].replace(app,"Other")

# Check to make sure binning was successful
lung_cancer_df['CHEST PAIN'].value_counts()

Other    309
Name: CHEST PAIN, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
print(lung_cancer_df['CHEST PAIN'].value_counts())

Other    309
Name: CHEST PAIN, dtype: int64


In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 500
classification_counts = lung_cancer_df['SHORTNESS OF BREATH'].value_counts()
classifications_to_replace = list(classification_counts[classification_counts < cutoff].index)

# Replace in dataframe
for cls in classifications_to_replace:
    lung_cancer_df['SHORTNESS OF BREATH'] = lung_cancer_df['SHORTNESS OF BREATH'].replace(cls,"Other")

# Check to make sure binning was successful
lung_cancer_df['SHORTNESS OF BREATH'].value_counts()

Other    309
Name: SHORTNESS OF BREATH, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
lung_cancer_df = pd.get_dummies(lung_cancer_df)

In [9]:
# Split our preprocessed data into our features and target arrays
X = lung_cancer_df.drop(columns=['CHRONIC DISEASE'])
y = lung_cancer_df['CHRONIC DISEASE']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               1500      
                                                                 
 dense_1 (Dense)             (None, 30)                3030      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 4,561
Trainable params: 4,561
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[tf.keras.callbacks.ModelCheckpoint(filepath='SurveyLungCancer.h5', verbose=1, save_weights_only=True, save_freq='epoch', period=5)])

Epoch 1/100
8/8 [==============================] - 3s 6ms/step - loss: 0.6202 - accuracy: 0.3074
Epoch 2/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0606 - accuracy: 0.4805
Epoch 3/100
8/8 [==============================] - 0s 6ms/step - loss: -0.4016 - accuracy: 0.4935
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: -0.8387 - accuracy: 0.4935
Epoch 5/100
1/8 [==>...........................] - ETA: 0s - loss: -1.0017 - accuracy: 0.5312
Epoch 5: saving model to SurveyLungCancer.h5
8/8 [==============================] - 0s 14ms/step - loss: -1.2743 - accuracy: 0.4935
Epoch 6/100
8/8 [==============================] - 0s 6ms/step - loss: -1.7402 - accuracy: 0.4935
Epoch 7/100
8/8 [==============================] - 0s 4ms/step - loss: -2.2656 - accuracy: 0.4935
Epoch 8/100
8/8 [==============================] - 0s 3ms/step - loss: -2.8867 - accuracy: 0.4935
Epoch 9/100
8/8 [==============================] - 0s 6ms/step - loss: -3.5888 - accuracy:

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3/3 - 1s - loss: -4.9168e+03 - accuracy: 0.5000 - 518ms/epoch - 173ms/step
Loss: -4916.77001953125, Accuracy: 0.5


In [15]:
# Export our model to HDF5 file
nn.save('SurveyLungCancer.h5')